In [145]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaModel
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm.notebook import tqdm
import math

In [146]:
'''
Dataset Class, returns standard inputs for transformer (primary data) and
ancillary_input (secondary data if training transformer-CNN)
'''

class TextDataset(Dataset):

    def __init__(self, train_data, labels, td_tokenizer, max_len, label_type):
        self.clinical_history = train_data['Clinical history'].values.tolist()
        self.ancillary_data = train_data.copy().drop([label_type, 'Clinical history'], axis=1).values.tolist()
        self.labels = labels
        self.td_tokenizer = td_tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.clinical_history)

    def __getitem__(self, idx):
        text = self.clinical_history[idx]
        label = self.labels[idx]
        ancillary_data = self.ancillary_data[idx]
        encoding = self.td_tokenizer(text, return_tensors='pt', max_length=self.max_len, padding='max_length',
                                  truncation=True, add_special_tokens=True)

        return {'input_ids': encoding['input_ids'].unsqueeze(0).flatten(),
                'attention_mask': encoding['attention_mask'].unsqueeze(0).flatten(),
                'label': torch.tensor(label),
                'ancillary_input': torch.tensor(ancillary_data).unsqueeze(0).flatten()}


In [147]:
'''
RoBERTa classifier (no CNN) - primary clinical history data only
'''
class RobertaClassifier(nn.Module):

    def __init__(self, r_model, classes):
        super(RobertaClassifier, self).__init__()
        self.roberta = RobertaForSequenceClassification.from_pretrained(r_model, num_labels=classes)

    def forward(self, input_ids, attention_mask, labels):
        return self.roberta(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

In [148]:
'''
Combined RoBERTa classifier and CNN
r_model: RoBERTa model name, classes: number of classes, feature_len: length of ancillary data
'''
class RobertaCNNClassifier(nn.Module):

    def __init__(self, r_model, classes, feature_len):
        super(RobertaCNNClassifier, self).__init__()
        self.comb_len = feature_len + 768
        self.roberta = RobertaModel.from_pretrained(r_model)
        self.pre_conv = nn.Linear(self.comb_len, self.comb_len)

        # Convolutional layers
        self.conv1d_1 = nn.Conv1d(1, 16, kernel_size=3)
        self.conv1d_2 = nn.Conv1d(16, 32, kernel_size=3)
        self.conv1d_3 = nn.Conv1d(32, 16, kernel_size=3)

        # Dropout, MaxPool and Fully connected layers
        self.dropout = nn.Dropout(0.5)
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.pre_classifier = nn.Linear(2256, 1024)
        self.classifier = nn.Linear(1024, classes)

    def forward(self, input_ids, attention_mask, labels, ancillary_data):
        output1 = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output1.last_hidden_state
        hidden_state = hidden_state[:, 0, :]
        x = torch.concat((hidden_state, ancillary_data), dim=1)  #concatenate transformer output and ancillary ohe data
        x = self.pre_conv(x)  # batch_size=32, inputs=comb_len
        x = x.unsqueeze(2)
        x = x.permute(0, 2, 1)  # batch_size=32, channels=1, length=comb_len

        x = self.conv1d_1(x)
        x = nn.ReLU()(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        x = self.conv1d_2(x)
        x = nn.ReLU()(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        x = self.conv1d_3(x)
        x = nn.ReLU()(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        x = x.view(x.size(0), -1)  # re-shape to batch_size, channels x length
        x = self.pre_classifier(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        output = self.classifier(x)
        return output


In [149]:
'''
BERT classifier (no CNN)
bert_model: BERT model name, classes: number of classes
'''
class BERTClassifier(nn.Module):

    def __init__(self, bert_model, classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained(bert_model, num_labels=classes)

    def forward(self, input_ids, attention_mask, labels):
        return self.bert(input_ids=input_ids, attention_mask=attention_mask, labels=labels)


In [150]:
'''
Combined BERT classifier and CNN
bert_model: BERT model name, classes: number of classes, feature_len: length of ancillary data
'''
class BERTCNNClassifier(nn.Module):

    def __init__(self, bert_model, classes, feat_len):
        super(BERTCNNClassifier, self).__init__()
        self.comb_len = feat_len + 768
        self.bert = BertModel.from_pretrained(bert_model)
        self.pre_conv = nn.Linear(self.comb_len, self.comb_len)

        # Convolutional layers
        self.conv1d_1 = nn.Conv1d(1, 16, kernel_size=3)
        self.conv1d_2 = nn.Conv1d(16, 32, kernel_size=3)
        self.conv1d_3 = nn.Conv1d(32, 16, kernel_size=3)
        self.conv1d_4 = nn.Conv1d(512, 256, kernel_size=1)

        # Dropout, MaxPool and Fully connected layers
        self.dropout = nn.Dropout(0.5)
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.pre_classifier = nn.Linear(2256, 1024)
        self.classifier = nn.Linear(1024, classes)

    def forward(self, input_ids, attention_mask, labels, ancillary_data):
        output1 = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output1.last_hidden_state
        hidden_state = hidden_state[:, 0, :]
        x = torch.concat((hidden_state, ancillary_data), dim=1)  #concatenate transformer output and ancillary ohe data
        x = self.pre_conv(x)  # batch_size=32, inputs=comb_len
        x = x.unsqueeze(2)
        x = x.permute(0, 2, 1)  # batch_size=32, channels=1, length=comb_len

        x = self.conv1d_1(x)
        x = nn.ReLU()(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        x = self.conv1d_2(x)
        x = nn.ReLU()(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        x = self.conv1d_3(x)
        x = nn.ReLU()(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        x = x.view(x.size(0), -1)  # re-shape to batch_size, channels x length
        x = self.pre_classifier(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        output = self.classifier(x)
        return output

In [151]:
'''
Focal loss function, with hyperparameters alpha and gamma as described in Methods section.
 '''

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, weights=None):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.weights = weights

    def forward(self, inputs, targets):
        log_prob = F.log_softmax(inputs, dim=-1)
        pt = torch.exp(log_prob)

        if self.weights is not None:
            loss = F.nll_loss(
                (-self.alpha * (1 - pt) ** self.gamma) * log_prob,
                targets,
                weight=self.weights.float(),
                reduction = 'none')
        else:
            loss = F.nll_loss(
                ((1 - pt) ** self.gamma) * log_prob,
                targets,
                reduction = 'none')
        return loss.mean()

In [152]:
def predict(predict_model, test_data_loader, predict_device, label_type, cnn=None):
    predict_model.eval()
    y_pred = []
    y_true = []

    # Deactivate autograd engine to reduce memory usage
    # with torch.no_grad():
    with torch.inference_mode():

        for batch in tqdm(test_data_loader):
            # Pass tensors to device, includes ancillary secondary data if using Roberta-CNN classifier
            input_ids = batch['input_ids'].to(predict_device)
            attention_mask = batch['attention_mask'].to(predict_device)
            labels = batch['label'].to(predict_device)
            if cnn:
                ancillary_input = batch['ancillary_input'].to(predict_device)

            # Forward pass, includes ancillary secondary data if using Roberta-CNN classifier
            if cnn:
                outputs = predict_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, ancillary_data=ancillary_input)
            else:
                outputs = predict_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                outputs = outputs.logits

            y_pred.extend(outputs.cpu().tolist())
            y_true.extend(labels.cpu().tolist())

    if label_type == 'UrgBin':
        y_true = np.argmax(y_true, axis=1)
    y_pred = np.argmax(y_pred, axis=1)

    return y_pred, y_true


**Set Parameters**

In [153]:
'''
SET PARAMETERS
Load training and combined validation-test datasets from excel.
Note this was previously split to allow data augmentation of training data only.
set_label: 'Protocol' or 'UrgBin'
set_cnn: True will apply transformer-CNN, False will apply transformer only
set_model: 'roberta' or 'bert'
dir_path: Please replace this with the file path to the submission folder.
'''

set_label = 'Protocol'
set_cnn = True
set_model = 'bert'

dir_path = "/content/drive/MyDrive/Colab Notebooks/Final Notebooks/Submission/"
label_list = ['ID', 'Urg', 'UrgBin', 'Protocol']

X_train_df = pd.read_csv(dir_path + "/Data/Xtrain_aug_1000-400.csv")
y_train_df = X_train_df[set_label]

X_val_test_df = pd.read_csv(dir_path + "/Data/Xtest_aug_1000-400.csv")
y_val_test_df = X_val_test_df[set_label]

label_list.remove(set_label)
X_train_df = X_train_df.drop(label_list, axis=1)
X_val_test_df = X_val_test_df.drop(label_list, axis=1)

In [154]:
print(X_val_test_df.head().to_string())
print(y_val_test_df.head().to_string())


                                                                                                                                                                                                                                                                                                                                                                                                                                      Clinical history  Protocol       Age  eGFR_1  eGFR_2  eGFR_3a  eGFR_3b  eGFR_4  eGFR_5  eGFR_None  Referring Locat_5FL-PA WD MAIN  Referring Locat_5K4-CH WD MULB  Referring Locat_BEMS PLUS  Referring Locat_BREAST RUH  Referring Locat_CARD ECHO RUH  Referring Locat_CARDIAC WD  Referring Locat_CARDIO RUH  Referring Locat_CHEMO RUH  Referring Locat_DERM RUH  Referring Locat_DIAB RUH  Referring Locat_ENDO RUH  Referring Locat_ENT RUH  Referring Locat_EYES RUH  Referring Locat_FRACTURE RUH  Referring Locat_GP  Referring Locat_GYNAE EPAC  Referring Locat_GYNAE RUH  Referring Lo

In [155]:
# Get dimensions of all combined features - Protocol and ancillary
feat_len = X_train_df.copy().drop(['Clinical history', set_label], axis=1).shape[1]

In [156]:
# Train, validation, test split
X_val_df, X_test_df, y_val_df, y_test_df = train_test_split(X_val_test_df, y_val_test_df, test_size=0.33,
                                                            random_state=42)

y_train = y_train_df.values.tolist()
y_val = y_val_df.values.tolist()
y_test = y_test_df.values.tolist()

In [157]:
# If urgency classification, sets number of classes, criterion to BCEWithLogitsLoss and converts y labels to suitable format
if set_label == 'UrgBin':
    num_classes = 2
    y_test = [[float(i), 1.0-i] for i in y_test]
else:
    num_classes = max(y_train) + 1

# Set Transformer hyperparameters, see manuscript for optimal settings
# max_sequence_length optimised at 256, note 512 not feasible for all models due to GPU constraints

batch_size = 16
max_sequence_length = 256


In [158]:
# Set up testing model as roberta or bert
if set_model == 'roberta':
    model_name = dir_path + 'RoBERTa-base-PM-M3-Voc-distill-align-hf'
    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    if set_label == 'UrgBin':
        model_path = dir_path + "Models/urgency_roberta.pth"
    else:
        model_path = dir_path + "Models/protocol_roberta.pth"

elif set_model == 'bert':
    model_name = 'emilyalsentzer/Bio_ClinicalBERT'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    if set_label == 'UrgBin':
        model_path = dir_path + "Models/urgency_bert.pth"
    else:
        model_path = dir_path + "Models/protocol_bert.pth"

if set_cnn:
    model_path = model_path.replace('.pth', '_cnn.pth')

# Initialise test datasets and dataloader
test_dataset = TextDataset(X_test_df, y_test, tokenizer, max_sequence_length, set_label)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [159]:
# Set device, model, optimizer, warmup, scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if set_model == 'roberta':
    if set_cnn:
        # Load previously trained RoBERTa model
        model = RobertaCNNClassifier(model_name, num_classes, feat_len).to(device)
        model.load_state_dict(torch.load(model_path))
    else:
        model = RobertaClassifier(model_name, num_classes).to(device)
        model.load_state_dict(torch.load(model_path))

elif set_model == 'bert':
    # if set_cnn, loads previous weights for previously trained model
    if set_cnn:
        model = BERTCNNClassifier(model_name, num_classes, feat_len).to(device)
        model.load_state_dict(torch.load(model_path))
    else:
        model = BERTClassifier(model_name, num_classes).to(device)
        model.load_state_dict(torch.load(model_path))

<ipython-input-159-d7bb1b1ae1fd>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


In [160]:
# Set seed value
seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
print(model)

BERTCNNClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [161]:
test_pred, test_true = predict(model, test_dataloader, device, set_label, cnn=set_cnn)
classification = classification_report(test_true, test_pred)
print(f"Test Classification:\n{classification}")


  0%|          | 0/147 [00:00<?, ?it/s]

Test Classification:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.95      0.95      0.95        21
           2       0.00      0.00      0.00         2
           3       0.60      0.34      0.43        53
           4       0.25      0.33      0.29         3
           5       0.75      0.50      0.60         6
           6       1.00      0.50      0.67         2
           7       0.20      0.25      0.22         4
           8       0.88      0.58      0.70        12
           9       0.88      0.93      0.91       213
          10       0.67      0.50      0.57        12
          11       0.67      0.75      0.71        16
          12       0.00      0.00      0.00         2
          13       0.25      0.33      0.29         3
          14       0.50      0.47      0.48        17
          15       0.58      0.30      0.39        64
          16       0.00      0.00      0.00         1
      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [162]:
#from google.colab import runtime
#runtime.unassign()